# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import glob
import keras
import IPython.display as ipd
import plotly.graph_objs as go
import plotly.offline as py
import plotly.tools as tls
import seaborn as sns
import pickle
import sklearn

In [ ]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)
from sklearn.model_selection import RandomizedSearchCV
# Importing Classifier Modules
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Preprocessing

In [ ]:
raw_csv_data=pd.read_csv("/content/drive/MyDrive/datasetW4/estimated_days.csv")
df=raw_csv_data.copy()
df

,id,origin,destination,service,days,deleted,date_entered,date_modified
0,59554E02-4C5E-4DA6-893A-00553CF34DBE,Christchurch,Christchurch,Point to Point,1,False,NaN,NaN
1,74811D3C-BA5E-4301-8F47-00690083AAEC,Auckland,Wellington,Chilled,2,False,NaN,NaN
2,AF2439BE-A452-41EE-98D5-007D0AD28220,Gisborne,Greymouth,Economy,6,False,NaN,NaN
3,606081E8-4A47-40F1-A409-00B6CD546A97,Mosgiel,Christchurch,Economy,3,False,NaN,NaN
4,FAD3527E-3373-4965-B324-036038981811,Auckland,Dargaville,Economy,1,False,NaN,NaN
...,...,...,...,...,...,...,...,...
824,75C8C740-CD9A-4F57-BC28-FEED690891B0,Auckland,Nelson airport,Economy,2,False,NaN,NaN
825,52A55C8A-324F-4ADF-BD18-FF02067FD75A,Auckland,Dunedin,Economy,3,False,NaN,NaN
826,6829AE18-0320-4B56-98ED-FF1B404F9715,Auckland,Glentunnel,Express - Road,4,False,NaN,NaN
827,1CC49B7B-4F00-4A6D-90DB-FF5D275497D3,Auckland,Cardrona,Economy,2,False,NaN,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829 entries, 0 to 828
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             829 non-null    object 
 1   origin         829 non-null    object 
 2   destination    829 non-null    object 
 3   service        829 non-null    object 
 4   days           829 non-null    int64  
 5   deleted        829 non-null    bool   
 6   date_entered   0 non-null      float64
 7   date_modified  4 non-null      object 
dtypes: bool(1), float64(1), int64(1), object(5)
memory usage: 46.3+ KB


In [ ]:
pd.set_option('display.max_rows', None)
df['days'].value_counts()

2     233
1     214
3     192
4      83
5      51
6      19
7      11
8       6
9       3
10      3
18      2
13      2
14      1
21      1
23      1
25      1
15      1
39      1
31      1
11      1
56      1
41      1
Name: days, dtype: int64

### Data Cleaning

In [ ]:
Ddata = df.copy()
Ddata=pd.concat([Ddata['origin'],Ddata['destination'],Ddata['service'],Ddata['days']], axis=1)
Ddata.head(10)

,origin,destination,service,days
0,Christchurch,Christchurch,Point to Point,1
1,Auckland,Wellington,Chilled,2
2,Gisborne,Greymouth,Economy,6
3,Mosgiel,Christchurch,Economy,3
4,Auckland,Dargaville,Economy,1
5,Auckland,Ruatoria,Economy,4
6,Christchurch,Invercargill,Economy,1
7,Auckland,Christchurch,Chilled,3
8,Ikamatua,Papamoa beach,Economy,3
9,Christchurch,Heathcote valley,Metro,1


In [ ]:
Ddata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829 entries, 0 to 828
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   origin       829 non-null    object
 1   destination  829 non-null    object
 2   service      829 non-null    object
 3   days         829 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 26.0+ KB


In [ ]:
Ddata['origin'] = Ddata['origin'].str.lower()
Ddata['destination'] = Ddata['destination'].str.lower()
Ddata['service'] = Ddata['service'].str.lower()

Ddata['origin'] = Ddata['origin'].str.strip()
Ddata['destination'] = Ddata['destination'].str.strip()
Ddata['service'] = Ddata['service'].str.strip()
Ddata.head(10)

,origin,destination,service,days
0,christchurch,christchurch,point to point,1
1,auckland,wellington,chilled,2
2,gisborne,greymouth,economy,6
3,mosgiel,christchurch,economy,3
4,auckland,dargaville,economy,1
5,auckland,ruatoria,economy,4
6,christchurch,invercargill,economy,1
7,auckland,christchurch,chilled,3
8,ikamatua,papamoa beach,economy,3
9,christchurch,heathcote valley,metro,1


In [ ]:
df_preprocessed=Ddata.copy()
df_preprocessed.to_csv('deliveryDate.csv', index=False)

### Mapping

In [ ]:
with open('/content/drive/MyDrive/datasetW4/cityDict3.pkl', 'rb') as f:
    cityDict = pickle.load(f)
with open('/content/drive/MyDrive/datasetW4/serviceDict.pkl', 'rb') as f:
    serviceDict = pickle.load(f)

In [ ]:
Ddata= Ddata.replace(to_replace="express - air", value="express-air")

In [ ]:
Ddata= Ddata.replace(to_replace="metro", value="metro general")

In [ ]:
DateData = Ddata.copy()
DateData2 = Ddata.copy()

In [ ]:
DateData['origin']=DateData['origin'].map(cityDict)
DateData['destination']=DateData['destination'].map(cityDict)
DateData['service']=DateData['service'].map(serviceDict)
DateData.head(10)

,origin,destination,service,days
0,434,434,3,1
1,50,465,7,2
2,441,466,2,6
3,241,434,2,3
4,50,390,2,1
5,50,546,2,4
6,434,287,2,1
7,50,434,7,3
8,258,531,2,3
9,434,547,5,1


In [ ]:
DateData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829 entries, 0 to 828
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   origin       829 non-null    int64
 1   destination  829 non-null    int64
 2   service      829 non-null    int64
 3   days         829 non-null    int64
dtypes: int64(4)
memory usage: 26.0 KB


In [ ]:
print(DateData['origin'].isnull().sum())
print(DateData['destination'].isnull().sum())
print(DateData['service'].isnull().sum())

0
0
0


In [ ]:
df_preprocessed=DateData.copy()
df_preprocessed.to_csv('deliveryDateMapped.csv', index=False)

### New City Dictionary

In [ ]:
o = Ddata[DateData['origin'].isnull()]
d = Ddata[DateData['destination'].isnull()]

In [ ]:
o['origin'].unique()

array(['papamoa beach', 'port nelson', 'dunedin north', 'port taranaki',
       'bridge hill', 'dunedin south', 'dunedin airport', 'meremere',
       'port whangarei', 'nelson airport', 'invercargill west',
       'timaru port', 'arthurs point', 'lake pukaki', 'freshford'],
      dtype=object)

In [ ]:
d['destination'].unique()

array(['ruatoria', 'papamoa beach', 'heathcote valley', 'timaru central',
       'cape foulwind', 'east gore', 'tapawera', 'maitai',
       'port taranaki', 'maori hill', 'port whangarei', 'oakura', 'stoke',
       'parkside', 'awanui', 'concord', 'hornby south', 'bradford',
       'flaxton', 'invercargill airport', 'north loburn', 'nelson south',
       'arthurs point', 'port nelson', 'great barrier island', 'tapanui',
       'greenfield', 'bridge hill', 'nelson central', 'dunedin airport',
       'tinwald', 'mayfield', 'nelson airport', 'dalefield',
       'mangawhai heads', 'dunedin south', 'weedons', 'whangarei central',
       'tinopai', 'cheviot', 'selwyn', 'ohinewai', 'hawarden',
       'green island', 'atiamuri', 'timaru port', 'wairakei', 'pauanui',
       'burkes pass', 'franz josef glacier', 'pipitea', 'wakatu',
       'national park', 'earnscleugh', 'gladstone', 'mahana', 'wyndham',
       'waipa', 'kainga', 'little river', 'halfway bush', 'takapu valley',
       'bishopdal

In [ ]:
new_cities = pd.concat([o['origin'],d['destination']], axis=0)
new_cities

In [ ]:
new_cities2=new_cities.unique()

In [ ]:
import csv

file = open("/content/drive/MyDrive/datasetW4/city_updated.csv", "r")
data = list(csv.reader(file, delimiter=","))
file.close()

In [ ]:
citylist= list()
for i in data:
  for j in i:
    #print(j)
    citylist.append(str(j).lower())

In [ ]:
for i in new_cities2:
  citylist.append(str(i).lower())

In [ ]:
c=1
cityDict = {}
cityDict = {citylist[i]: int(i+1) for i in range(len(citylist))}
cityDict

{'hastings': 1,
 'cable bay': 2,
 'baylys beach': 3,
 'aniseed valley': 4,
 'otanomomo': 5,
 'tirau': 6,
 'eltham': 7,
 'te teko': 8,
 'lake hayes': 9,
 'mangatawhiri': 10,
 'matapouri': 11,
 'tekapo': 12,
 'balcairn': 13,
 'ōpōtiki': 14,
 'oamaru': 15,
 'opotiki': 16,
 'paihia': 17,
 'bunnythorpe': 18,
 'franz josef': 19,
 'pleasant valley': 20,
 'kaingaroa forest': 21,
 'levels': 22,
 'brooklyn': 23,
 'teviotdale': 24,
 'ormond': 25,
 'silverdale': 26,
 'kawakawa': 27,
 'ashhurst': 28,
 'eskdale': 29,
 'haumoana': 30,
 'southbridge': 31,
 'hikurangi': 32,
 'waihi': 33,
 'albert town': 34,
 'bannockburn': 35,
 'wellsford': 36,
 'murupara': 37,
 'reefton': 38,
 'morven': 39,
 'pokeno': 40,
 'pōkeno': 41,
 'waimauku': 42,
 'aokautere': 43,
 'whanagrei heads': 44,
 'montalto': 45,
 'pipiwai': 46,
 'fairview': 47,
 'west eyreton': 48,
 'little wanganui': 49,
 'auckland': 50,
 'whangarei': 51,
 'whangārei': 52,
 'waipukurau': 53,
 'branxholme': 54,
 'orari': 55,
 'pohuehue': 56,
 'hapuku':

In [ ]:
with open('cityDict3.pkl', 'wb') as f:
    pickle.dump(cityDict, f)

with open('cityDict3.pkl', 'rb') as f:
    cityDict = pickle.load(f)

# **Training Testing**

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(DateData.iloc[:,:-1].values, DateData['days'].values, test_size=0.2, random_state=0)

In [ ]:
train,test=train_test_split(DateData, test_size=0.2, random_state=0)

In [ ]:
X_train2,X_test2,y_train2,y_test2 = X_train,X_test,y_train,y_test

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(663, 3)
(663,)
(166, 3)
(166,)


##**XG BOOST**

In [ ]:
import xgboost as xgb
from xgboost.sklearn import XGBRegressor

xgb_model = XGBRegressor(verbosity=0)
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

result2 = xgb_model.score(y_test,y_pred)
print('Accuracy:',result2*100,'%')
#print(confusion_matrix(y_test.values, y_pred))

In [ ]:
y_test2 = pd.DataFrame(y_test, columns = ['days'])
y_pred2 = pd.DataFrame(y_pred, columns = ['days'])
mse=mean_squared_error(y_test2, y_pred2)

print(np.sqrt(mse))

2.9387176032859883


In [ ]:
y_pred2['days']=y_pred2['days'].apply(np.int64)

In [ ]:
y_pred2

In [ ]:
result2 = accuracy_score(y_test2,y_pred2)
print('Accuracy:',result2*100,'%')

Accuracy: 27.10843373493976 %


In [ ]:
y_test

array([ 4,  2,  3,  1,  4,  3, 25,  1,  1,  3,  4,  2,  1,  4,  1,  1,  4,
        7,  1,  2,  5,  3,  2,  1,  3,  1,  4,  1,  2,  3,  6,  3,  4,  4,
        3,  1,  5,  3,  5,  3,  4,  1,  1,  5,  3,  2,  3,  4,  3,  3,  1,
        2,  1,  2,  1,  2,  3,  3,  3,  2,  1,  1,  3,  1,  2,  1,  1,  1,
        1,  3,  3,  1,  2,  4,  1,  4,  4,  6,  1,  5,  5,  4,  1,  3,  2,
        3, 23,  5,  2,  5,  2,  3,  3,  5,  4,  4,  1,  3,  2,  2,  4,  2,
        5,  2,  3,  1,  3,  2,  3,  1,  3,  3,  1,  1,  4,  4,  1,  3,  4,
        8,  1,  1,  7,  4,  1,  1,  2,  3,  5,  2,  1,  2,  2,  4,  2,  1,
        2,  1,  1,  3,  2,  3,  2,  4,  3,  1,  1,  5,  4,  3,  2,  2,  6,
        2,  2,  4,  1,  5,  4,  2,  4,  2,  1,  1,  3,  3])

In [ ]:
y_pred

array([ 2.5081334,  2.5181484,  3.5230203,  2.4925885,  2.462274 ,
        2.645768 ,  2.5745366,  2.7582426,  2.3559985,  5.3242335,
        2.8444984,  2.3531382,  2.6877496,  2.5905561,  3.5307453,
        2.462274 ,  2.4330502,  3.475588 ,  2.3531382,  2.5905561,
        2.6885252,  2.6240299,  2.462274 ,  2.0911613,  2.4939704,
        2.5745366, 11.251157 ,  2.9265733,  2.7018137,  3.688685 ,
        3.9914467,  2.5481393,  3.8327417,  2.7599175,  2.922406 ,
        2.0911613,  2.9449234,  2.6457355,  7.0858717,  3.0655994,
        2.7049427,  1.7988107,  1.8868957,  5.1572514,  2.5576506,
        1.884259 ,  2.8482537,  3.8327417,  4.0963717,  2.8482537,
        2.4123683,  3.1688273,  2.758327 ,  2.196472 ,  2.2884426,
        2.6877496,  3.5265617,  3.1676116,  6.2411017,  2.8574312,
        2.4871492,  2.1960883,  2.7582426,  2.6775217,  2.3564734,
        2.7395234,  2.223796 ,  3.5913134,  5.7421293,  5.037955 ,
        3.332007 ,  3.332007 ,  2.5081334,  3.2040877,  2.7557

In [ ]:
cols_when_model_builds = xgb_model.get_booster().feature_names

In [ ]:
cols_when_model_builds

['f0', 'f1', 'f2']

##**CAT BOOST**

In [ ]:
from catboost import CatBoostRegressor
model = CatBoostRegressor(iterations=2,
                          learning_rate=1,
                          depth=2)
# Fit model
model.fit(X_train, y_train)
# Get predictions
preds = model.predict(X_test)
result2 = accuracy_score(y_test,preds)
print('Accuracy:',result2*100,'%')

In [ ]:
clf = CatBoostClassifier(verbose=0, n_estimators=100)#,feature_weights =[1.5,1,1,2])
clf.fit(X_train,y_train)
y_pred = clf.predict_proba(X_test)
y_pred2 = clf.predict(X_test)
result2 = accuracy_score(y_test,y_pred2)
print('Accuracy:',result2*100,'%')

np.set_printoptions(threshold=200)
result = confusion_matrix(y_test, y_pred2)
print('\nConfusion Matrix:')
print(result)

Accuracy: 27.10843373493976 %

Confusion Matrix:
[[18 19  7  2  0  0  0  0  0  0  0]
 [ 7 18  6  3  0  0  0  0  0  0  0]
 [ 4 23  6  2  0  1  0  1  1  0  0]
 [ 6 12  7  1  1  0  0  0  0  0  0]
 [ 2  5  3  0  2  1  0  0  0  0  0]
 [ 0  1  1  1  0  0  0  0  0  0  0]
 [ 1  1  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  0  0  0  0]]


##**KNN**

In [ ]:
from sklearn.model_selection import GridSearchCV

#List Hyperparameters that we want to tune.
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]

#Convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)

#Create new KNN object
knn_2 = KNeighborsClassifier()

#Use GridSearch
clf = GridSearchCV(knn_2, hyperparameters, cv=10)

#Fit the model
best_model = clf.fit(X_train,y_train)

#Print The value of best Hyperparameters
print('Best leaf_size:', best_model.best_estimator_.get_params()['leaf_size'])
print('Best p:', best_model.best_estimator_.get_params()['p'])
print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning:

The least populated class in y has only 1 members, which is less than n_splits=10.



Best leaf_size: 1
Best p: 2
Best n_neighbors: 21


In [ ]:
#Import knearest neighbors Classifier model
from sklearn.neighbors import KNeighborsClassifier


#Create KNN Classifier
knn = KNeighborsClassifier(leaf_size=1, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=21, p=2,
                     weights='uniform')

#Train the model using the training sets
knn.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = knn.predict(X_test)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.3072289156626506


# **New Approach from Status**

In [ ]:
raw_csv_data=pd.read_csv("/content/drive/MyDrive/datasetW4/delivery_new_data.csv")
df=raw_csv_data.copy()
df.head(5)

,consignment_id,consignment_note,Account_name,account_type,ConCreated,InActive,StatusName,StatusDateCreated,SenderCity,ReceiverCity,SenderSuburb,ReceiverSuburb,ServiceType
0,F4DC655C-C957-49DE-9572-479D357FA721,IFL3026283,BELGOTEX NZ LTD,Customer,2022-09-21,False,Unallocated,2022-09-21,Auckland,New Plymouth,MANGERE,NaN,SuperEconomy
1,3BEB434F-5D73-41B9-9FAC-1A6C648B8B82,ECC0040986,Ryco Dies Ltd,Customer,2022-01-24,False,Hold,2022-01-24,Auckland,Hastings,Wiri,Woolwich,Economy
2,4A7ECFDC-3A51-49A5-AB6D-DCB8C5359C5C,IFL9024720,BELGOTEX NZ LTD,Customer,2022-05-12,False,Unallocated,2022-05-12,Auckland,auckland,MANGERE,PUKEKOHE,SuperEconomy
3,E05739FB-FF24-498C-A6A7-423C74E7DFF9,IFL3021677,BELGOTEX NZ LTD,Customer,2022-05-13,False,POD Received,2022-05-13,Auckland,Porirua,MANGERE,Porirua City Centre,SuperEconomy
4,ABD10286-617B-414C-9275-791C0D2EC3A4,IFL9026923,BELGOTEX NZ LTD,Customer,2022-11-08,False,Unallocated,2022-11-08,Auckland,Auckland,MANGERE,Albany,Metro General


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129898 entries, 0 to 129897
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   consignment_id     129898 non-null  object
 1   consignment_note   129898 non-null  object
 2   Account_name       129898 non-null  object
 3   account_type       129898 non-null  object
 4   ConCreated         129898 non-null  object
 5   InActive           129898 non-null  bool  
 6   StatusName         129898 non-null  object
 7   StatusDateCreated  129898 non-null  object
 8   SenderCity         129889 non-null  object
 9   ReceiverCity       129849 non-null  object
 10  SenderSuburb       121025 non-null  object
 11  ReceiverSuburb     90424 non-null   object
 12  ServiceType        129898 non-null  object
dtypes: bool(1), object(12)
memory usage: 12.0+ MB


In [ ]:
Ddata2 = df.copy()
Ddata2=pd.concat([Ddata2['consignment_note'],Ddata2['Account_name'],Ddata2['SenderCity'],Ddata2['ReceiverCity'],Ddata2['ServiceType'],Ddata2['StatusName'],Ddata2['StatusDateCreated']], axis=1)
Ddata2.head(5)

,consignment_note,Account_name,SenderCity,ReceiverCity,ServiceType,StatusName,StatusDateCreated
0,IFL3026283,BELGOTEX NZ LTD,Auckland,New Plymouth,SuperEconomy,Unallocated,2022-09-21
1,ECC0040986,Ryco Dies Ltd,Auckland,Hastings,Economy,Hold,2022-01-24
2,IFL9024720,BELGOTEX NZ LTD,Auckland,auckland,SuperEconomy,Unallocated,2022-05-12
3,IFL3021677,BELGOTEX NZ LTD,Auckland,Porirua,SuperEconomy,POD Received,2022-05-13
4,IFL9026923,BELGOTEX NZ LTD,Auckland,Auckland,Metro General,Unallocated,2022-11-08


In [ ]:
Ddata2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129898 entries, 0 to 129897
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   consignment_note   129898 non-null  object
 1   Account_name       129898 non-null  object
 2   SenderCity         129889 non-null  object
 3   ReceiverCity       129849 non-null  object
 4   ServiceType        129898 non-null  object
 5   StatusName         129898 non-null  object
 6   StatusDateCreated  129898 non-null  object
dtypes: object(7)
memory usage: 6.9+ MB


In [ ]:
Ddata2['Account_name'] = Ddata2['Account_name'].str.lower()
Ddata2['SenderCity'] = Ddata2['SenderCity'].str.lower()
Ddata2['ReceiverCity'] = Ddata2['ReceiverCity'].str.lower()
Ddata2['ServiceType'] = Ddata2['ServiceType'].str.lower()
Ddata2['StatusName'] = Ddata2['StatusName'].str.lower()
Ddata2['StatusDateCreated'] = Ddata2['StatusDateCreated'].str.lower()

Ddata2['consignment_note'] = Ddata2['consignment_note'].str.strip()
Ddata2['Account_name'] = Ddata2['Account_name'].str.strip()
Ddata2['SenderCity'] = Ddata2['SenderCity'].str.strip()
Ddata2['ReceiverCity'] = Ddata2['ReceiverCity'].str.strip()
Ddata2['ServiceType'] = Ddata2['ServiceType'].str.strip()
Ddata2['StatusName'] = Ddata2['StatusName'].str.strip()
Ddata2['StatusDateCreated'] = Ddata2['StatusDateCreated'].str.strip()

Ddata2.head(10)

,consignment_note,Account_name,SenderCity,ReceiverCity,ServiceType,StatusName,StatusDateCreated
0,IFL3026283,belgotex nz ltd,auckland,new plymouth,supereconomy,unallocated,2022-09-21
1,ECC0040986,ryco dies ltd,auckland,hastings,economy,hold,2022-01-24
2,IFL9024720,belgotex nz ltd,auckland,auckland,supereconomy,unallocated,2022-05-12
3,IFL3021677,belgotex nz ltd,auckland,porirua,supereconomy,pod received,2022-05-13
4,IFL9026923,belgotex nz ltd,auckland,auckland,metro general,unallocated,2022-11-08
5,ECC0042983,wucha wellington,auckland,wellington,supereconomy,cancelled,2022-04-25
6,IFL9025662,belgotex nz ltd,auckland,auckland,metro general,unallocated,2022-07-21
7,ECC0045151,nz retail equipment,christchurch,auckland,economy,unallocated,2022-06-30
8,ECC0047970,auslink logistics limited,christchurch,wanaka,economy,unallocated,2022-11-09
9,ECC0047879,dhl express new zealand ltd,cromwell,christchurch,express - road,unallocated,2022-11-07


In [ ]:
Ddata2['StatusName'].unique()

array(['unallocated', 'hold', 'pod received', 'cancelled', 'allocated',
       'invoiced', 'out for delivery', 'collected', 'awaiting pod',
       'not sell rating correct', 'in transit', 'cost needed',
       'delivered', 'in port', 'claim', 'failed delivery',
       'delivery depot', 'urgent', 'cost sell needed',
       'pre april 2021 sell match only', 'simons problem children',
       'no charge'], dtype=object)

In [ ]:
Ddata2 = Ddata2.drop(Ddata2[(Ddata2['StatusName'] != 'unallocated') & (Ddata2['StatusName'] != 'delivered')].index)

In [ ]:
Ddata2['StatusName'].value_counts()

delivered      13505
unallocated    10945
Name: StatusName, dtype: int64

In [ ]:
Ddata2['consignment_note'].value_counts()>=2

ECC0034657     True
ECC0034731     True
ECC0034204     True
ECC003632A     True
ECC0036300     True
              ...  
ECC0035714    False
ECC0035715    False
ECC0035717    False
ECC0035718    False
ECC0044345    False
Name: consignment_note, Length: 15695, dtype: bool

In [ ]:
a= Ddata2[Ddata2['consignment_note']=='ECC0045035']
a

,consignment_note,Account_name,SenderCity,ReceiverCity,ServiceType,StatusName,StatusDateCreated
66563,ECC0045035,ryco dies ltd,auckland,christchurch,economy,delivered,2022-06-29
119620,ECC0045035,ryco dies ltd,auckland,christchurch,economy,unallocated,2022-06-22


# **New Approach Table**

In [ ]:
raw_csv_data=pd.read_csv("/content/drive/MyDrive/datasetW4/for_delivery_date.csv")
df=raw_csv_data.copy()
df.head(5)

,consignment_id,consignment_note,Account_name,delivery_date,date_allocated,SenderCity,ReceiverCity,ServiceType
0,13086FE7-12A1-4467-964C-0000047EE392,IFL3022016,BELGOTEX NZ LTD,2022-02-03,2022-02-01,Auckland,Te Awamutu,BOP - Waikato
1,60351FC5-10B6-4497-B349-000225FB24D8,ECC0047429,freightplus,2022-10-16,2022-10-13,Auckland,Palmerston North,SuperEconomy
2,0AA9C307-3746-4302-A0B6-0004F0ACD64A,ECC0040335,AUSLINK LOGISTICS LIMITED,2021-11-26,2021-11-24,Christchurch,Christchurch,Metro General
3,E6A42B8A-B673-4B7B-8DFF-0005F5264B82,IFL3022422,BELGOTEX NZ LTD,2022-02-25,2022-02-23,Auckland,Wellington,SuperEconomy
4,45559152-2187-4C29-9EF5-0008C207BABF,ECC0039329,DHL EXPRESS - CHCH,2021-10-21,2021-10-20,Christchurch,Lake Hayes,Economy


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26236 entries, 0 to 26235
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   consignment_id    26236 non-null  object
 1   consignment_note  26236 non-null  object
 2   Account_name      26236 non-null  object
 3   delivery_date     26236 non-null  object
 4   date_allocated    26236 non-null  object
 5   SenderCity        26233 non-null  object
 6   ReceiverCity      26223 non-null  object
 7   ServiceType       26236 non-null  object
dtypes: object(8)
memory usage: 1.6+ MB


In [ ]:
Ddata2 = df.copy()
Ddata2=pd.concat([Ddata2['consignment_note'],Ddata2['Account_name'],Ddata2['SenderCity'],Ddata2['ReceiverCity'],Ddata2['ServiceType'],Ddata2['date_allocated'],Ddata2['delivery_date']], axis=1)

Ddata2.head(5)

,consignment_note,Account_name,SenderCity,ReceiverCity,ServiceType,date_allocated,delivery_date
0,IFL3022016,BELGOTEX NZ LTD,Auckland,Te Awamutu,BOP - Waikato,2022-02-01,2022-02-03
1,ECC0047429,freightplus,Auckland,Palmerston North,SuperEconomy,2022-10-13,2022-10-16
2,ECC0040335,AUSLINK LOGISTICS LIMITED,Christchurch,Christchurch,Metro General,2021-11-24,2021-11-26
3,IFL3022422,BELGOTEX NZ LTD,Auckland,Wellington,SuperEconomy,2022-02-23,2022-02-25
4,ECC0039329,DHL EXPRESS - CHCH,Christchurch,Lake Hayes,Economy,2021-10-20,2021-10-21


In [ ]:
Ddata2['Account_name'] = Ddata2['Account_name'].str.lower()
Ddata2['SenderCity'] = Ddata2['SenderCity'].str.lower()
Ddata2['ReceiverCity'] = Ddata2['ReceiverCity'].str.lower()
Ddata2['ServiceType'] = Ddata2['ServiceType'].str.lower()

Ddata2['consignment_note'] = Ddata2['consignment_note'].str.strip()
Ddata2['Account_name'] = Ddata2['Account_name'].str.strip()
Ddata2['SenderCity'] = Ddata2['SenderCity'].str.strip()
Ddata2['ReceiverCity'] = Ddata2['ReceiverCity'].str.strip()
Ddata2['ServiceType'] = Ddata2['ServiceType'].str.strip()
Ddata2['date_allocated'] = Ddata2['date_allocated'].str.strip()
Ddata2['delivery_date'] = Ddata2['delivery_date'].str.strip()

Ddata3 = Ddata2.copy()
Ddata2.head(5)

,consignment_note,Account_name,SenderCity,ReceiverCity,ServiceType,date_allocated,delivery_date
0,IFL3022016,belgotex nz ltd,auckland,te awamutu,bop - waikato,2022-02-01,2022-02-03
1,ECC0047429,freightplus,auckland,palmerston north,supereconomy,2022-10-13,2022-10-16
2,ECC0040335,auslink logistics limited,christchurch,christchurch,metro general,2021-11-24,2021-11-26
3,IFL3022422,belgotex nz ltd,auckland,wellington,supereconomy,2022-02-23,2022-02-25
4,ECC0039329,dhl express - chch,christchurch,lake hayes,economy,2021-10-20,2021-10-21


In [ ]:
from datetime import datetime
i=0
for date_str in Ddata2['date_allocated']:
  date_object = datetime.strptime(date_str, '%Y-%m-%d').date()
  Ddata2['date_allocated'][i] = date_object
  i=i+1

i=0
for date_str in Ddata2['delivery_date']:
  date_object = datetime.strptime(date_str, '%Y-%m-%d').date()
  Ddata2['delivery_date'][i] = date_object
  i=i+1

In [ ]:
Ddata2.head(5)

,consignment_note,Account_name,SenderCity,ReceiverCity,ServiceType,date_allocated,delivery_date
0,IFL3022016,belgotex nz ltd,auckland,te awamutu,bop - waikato,2022-02-01,2022-02-03
1,ECC0047429,freightplus,auckland,palmerston north,supereconomy,2022-10-13,2022-10-16
2,ECC0040335,auslink logistics limited,christchurch,christchurch,metro general,2021-11-24,2021-11-26
3,IFL3022422,belgotex nz ltd,auckland,wellington,supereconomy,2022-02-23,2022-02-25
4,ECC0039329,dhl express - chch,christchurch,lake hayes,economy,2021-10-20,2021-10-21


In [ ]:
Ddata4 = Ddata2.copy()
Ddata4.head(5)

,consignment_note,Account_name,SenderCity,ReceiverCity,ServiceType,date_allocated,delivery_date
0,IFL3022016,belgotex nz ltd,auckland,te awamutu,bop - waikato,2022-02-01,2022-02-03
1,ECC0047429,freightplus,auckland,palmerston north,supereconomy,2022-10-13,2022-10-16
2,ECC0040335,auslink logistics limited,christchurch,christchurch,metro general,2021-11-24,2021-11-26
3,IFL3022422,belgotex nz ltd,auckland,wellington,supereconomy,2022-02-23,2022-02-25
4,ECC0039329,dhl express - chch,christchurch,lake hayes,economy,2021-10-20,2021-10-21


In [ ]:
Ddata2['delivery_date'][3]

datetime.date(2022, 2, 25)

In [ ]:
from datetime import datetime


date_str = '2022-2-25'
date_object = datetime.strptime(date_str, '%Y-%m-%d').date()
print(type(date_object))
print(date_object)

<class 'datetime.date'>
2022-02-25


In [ ]:
date_object

datetime.date(2022, 2, 25)

In [ ]:
from datetime import date


d0 = date(2008, 8, 18)
d1 = date(2008, 9, 26)
delta = d1 - d0
print(delta.days)

39


In [ ]:
Ddata4['days']=Ddata4['delivery_date']-Ddata4['date_allocated']
Ddata4.head(15)

,consignment_note,Account_name,SenderCity,ReceiverCity,ServiceType,date_allocated,delivery_date,days
0,IFL3022016,belgotex nz ltd,auckland,te awamutu,bop - waikato,2022-02-01,2022-02-03,"2 days, 0:00:00"
1,ECC0047429,freightplus,auckland,palmerston north,supereconomy,2022-10-13,2022-10-16,"3 days, 0:00:00"
2,ECC0040335,auslink logistics limited,christchurch,christchurch,metro general,2021-11-24,2021-11-26,"2 days, 0:00:00"
3,IFL3022422,belgotex nz ltd,auckland,wellington,supereconomy,2022-02-23,2022-02-25,"2 days, 0:00:00"
4,ECC0039329,dhl express - chch,christchurch,lake hayes,economy,2021-10-20,2021-10-21,"1 day, 0:00:00"
5,IFL3021888,belgotex nz ltd,auckland,wellington,supereconomy,2022-01-21,2022-01-23,"2 days, 0:00:00"
6,IFL3024717,belgotex nz ltd,auckland,new plymouth,supereconomy,2022-07-01,2022-07-07,"6 days, 0:00:00"
7,IFL9023641,belgotex nz ltd,auckland,auckland,metro general,2022-02-02,2022-07-05,"153 days, 0:00:00"
8,IFL9024552,belgotex nz ltd,auckland,auckland,metro general,2022-04-26,2022-04-29,"3 days, 0:00:00"
9,IFL3023094,belgotex nz ltd,auckland,new plymouth,supereconomy,2022-04-05,2022-04-06,"1 day, 0:00:00"


In [ ]:
for i in range(len(Ddata4['days'])):
  a=Ddata4['days'][i].days
  print(a)
  Ddata4['days'][i]=a

In [ ]:
Ddata4.head(15)

,consignment_note,Account_name,SenderCity,ReceiverCity,ServiceType,date_allocated,delivery_date,days
0,IFL3022016,belgotex nz ltd,auckland,te awamutu,bop - waikato,2022-02-01,2022-02-03,2
1,ECC0047429,freightplus,auckland,palmerston north,supereconomy,2022-10-13,2022-10-16,3
2,ECC0040335,auslink logistics limited,christchurch,christchurch,metro general,2021-11-24,2021-11-26,2
3,IFL3022422,belgotex nz ltd,auckland,wellington,supereconomy,2022-02-23,2022-02-25,2
4,ECC0039329,dhl express - chch,christchurch,lake hayes,economy,2021-10-20,2021-10-21,1
5,IFL3021888,belgotex nz ltd,auckland,wellington,supereconomy,2022-01-21,2022-01-23,2
6,IFL3024717,belgotex nz ltd,auckland,new plymouth,supereconomy,2022-07-01,2022-07-07,6
7,IFL9023641,belgotex nz ltd,auckland,auckland,metro general,2022-02-02,2022-07-05,153
8,IFL9024552,belgotex nz ltd,auckland,auckland,metro general,2022-04-26,2022-04-29,3
9,IFL3023094,belgotex nz ltd,auckland,new plymouth,supereconomy,2022-04-05,2022-04-06,1


In [ ]:
len(Ddata4['days'])

26236

In [ ]:
type(Ddata4['days'][0])

int

In [ ]:
df_preprocessed=Ddata4.copy()
df_preprocessed.to_csv('dateData.csv', index=False)

In [ ]:
Ddata4=pd.read_csv("/content/drive/MyDrive/datasetW4/dateData.csv")

In [ ]:
Ddata4=Ddata4.drop(['consignment_note'], axis=1)
Ddata4=Ddata4.drop(['date_allocated'], axis=1)
Ddata4=Ddata4.drop(['delivery_date'], axis=1)
Ddata4=Ddata4.drop(['Account_name'], axis=1)

In [ ]:
Ddata4.head(5)

,SenderCity,ReceiverCity,ServiceType,days
0,auckland,te awamutu,bop - waikato,2
1,auckland,palmerston north,supereconomy,3
2,christchurch,christchurch,metro general,2
3,auckland,wellington,supereconomy,2
4,christchurch,lake hayes,economy,1


In [ ]:
with open('/content/drive/MyDrive/datasetW4/cityDict3.pkl', 'rb') as f:
    cityDict = pickle.load(f)
with open('/content/drive/MyDrive/datasetW4/serviceDict.pkl', 'rb') as f:
    serviceDict = pickle.load(f)

In [ ]:
Ddata5=Ddata4.copy()
Ddata4['SenderCity']=Ddata4['SenderCity'].map(cityDict)
Ddata4['ReceiverCity']=Ddata4['ReceiverCity'].map(cityDict)
Ddata4['ServiceType']=Ddata4['ServiceType'].map(serviceDict)
Ddata4.head(10)

,SenderCity,ReceiverCity,ServiceType,days
0,50.0,276.0,20,2
1,50.0,260.0,8,3
2,434.0,434.0,5,2
3,50.0,465.0,8,2
4,434.0,9.0,2,1
5,50.0,465.0,8,2
6,50.0,227.0,8,6
7,50.0,50.0,5,153
8,50.0,50.0,5,3
9,50.0,227.0,8,1


In [ ]:
Ddata4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26236 entries, 0 to 26235
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SenderCity    26233 non-null  float64
 1   ReceiverCity  26221 non-null  float64
 2   ServiceType   26236 non-null  int64  
 3   days          26236 non-null  int64  
dtypes: float64(2), int64(2)
memory usage: 820.0 KB


In [ ]:
Ddata5[Ddata4['SenderCity'].isnull()]

,SenderCity,ReceiverCity,ServiceType,days
2629,NaN,auckland,economy,2
2952,NaN,christchurch,container,0
16313,NaN,lower hutt,container,2


In [ ]:
Ddata4.drop(Ddata4.loc[Ddata4['SenderCity'].isnull()].index, inplace = True)
Ddata4.drop(Ddata4.loc[Ddata4['ReceiverCity'].isnull()].index, inplace = True)
Ddata4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26218 entries, 0 to 26235
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SenderCity    26218 non-null  float64
 1   ReceiverCity  26218 non-null  float64
 2   ServiceType   26218 non-null  int64  
 3   days          26218 non-null  int64  
dtypes: float64(2), int64(2)
memory usage: 1.0 MB


In [ ]:
df_preprocessed=Ddata4.copy()
df_preprocessed.to_csv('deliveryDateData.csv', index=False)

In [ ]:
DeliveryData=pd.read_csv("/content/drive/MyDrive/datasetW4/deliveryDateData.csv")
DeliveryData.head(5)

,SenderCity,ReceiverCity,ServiceType,days
0,50.0,276.0,20,2
1,50.0,260.0,8,3
2,434.0,434.0,5,2
3,50.0,465.0,8,2
4,434.0,9.0,2,1


In [ ]:
DeliveryData.drop(DeliveryData[DeliveryData['days'] > 15].index, inplace = True)
DeliveryData.drop(DeliveryData[DeliveryData['days'] < 0].index, inplace = True)

In [ ]:
DeliveryData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25561 entries, 0 to 26216
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SenderCity    25561 non-null  float64
 1   ReceiverCity  25561 non-null  float64
 2   ServiceType   25561 non-null  int64  
 3   days          25561 non-null  int64  
dtypes: float64(2), int64(2)
memory usage: 998.5 KB


In [ ]:
DeliveryData.sort_values(['days'], ascending=False)

In [ ]:
df_preprocessed=DeliveryData.copy()
df_preprocessed.to_csv('deliveryDateData.csv', index=False)

## **Training on new data**

In [ ]:
DeliveryData.head(5)

,SenderCity,ReceiverCity,ServiceType,days
0,50.0,276.0,20,2
1,50.0,260.0,8,3
2,434.0,434.0,5,2
3,50.0,465.0,8,2
4,434.0,9.0,2,1


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(DeliveryData.iloc[:,:-1], DeliveryData['days'], test_size=0.25, random_state=0)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(19170, 3)
(19170,)
(6391, 3)
(6391,)


**#XGBOOST**

In [ ]:
#XGBOOST
import xgboost as xgb
xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42,feature_weights =[3.5,3.5,1])
xgb_model.fit(X_train.values, y_train)

y_pred = xgb_model.predict(X_test.values)

result2 = accuracy_score(y_test.values,y_pred)
print('Accuracy:',result2*100,'%')
#print(confusion_matrix(y_test.values, y_pred))

Accuracy: 40.6039743389141 %


In [ ]:
# save the model to disk
import pickle
filename = 'xgboost_date.sav'
pickle.dump(xgb_model, open(filename, 'wb'))

# some time later...

# load the model from disk
from sklearn import metrics
loaded_model = pickle.load(open(filename, 'rb'))
result = metrics.accuracy_score(y_test, y_pred)
print(result)

0.406039743389141


In [ ]:
senderCity = 'CHRISTCHURCH'
receiverCity = 'auckland'
serviceType = 'metro general'

SenderCity = cityDict[senderCity.lower()]
ReceiverCity = cityDict[receiverCity.lower()]
ServiceType = serviceDict[serviceType.lower()]
print("senderCity:",SenderCity,"\nreceiverCity:",ReceiverCity,"\nserviceType:",ServiceType)

predicted= xgb_model.predict([[SenderCity, ReceiverCity, ServiceType]]) # 0:Overcast, 2:Mild
print("days: ",predicted[0])

senderCity: 434 
receiverCity: 50 
serviceType: 5
days:  2


**#CAT BOOST**

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weights = dict(zip(classes, weights))

#clf = CatBoostClassifier(verbose=0, n_estimators=100 , class_weights=class_weights)
#clf.fit(X_train,y_train)
clf = CatBoostClassifier(verbose=0,loss_function='MultiClass', class_weights=class_weights,feature_weights =[2.5,2.5,1])
clf.fit(X_train, y_train)

In [ ]:
y_pred=clf.predict(X_test)
result2 = accuracy_score(y_test,y_pred)
print('Accuracy:',result2*100,'%')

np.set_printoptions(threshold=200)
result = confusion_matrix(y_test, y_pred)
print('\nConfusion Matrix:')
print(result)


Accuracy: 23.645609231371015 %

Confusion Matrix:
[[ 27 135  27 ...   8   2   4]
 [  6 700  30 ...   1   2   3]
 [ 47 352 304 ...  53  20  10]
 ...
 [  0   0   0 ...   0   0   0]
 [  1   0   0 ...   0   0   1]
 [  0   0   0 ...   0   1   1]]


In [ ]:
clf = CatBoostClassifier(verbose=0, n_estimators=100,feature_weights =[3.5,3.5,1])
clf.fit(X_train,y_train)
y_pred = clf.predict_proba(X_test)
y_pred2 = clf.predict(X_test)
result2 = accuracy_score(y_test,y_pred2)
print('Accuracy:',result2*100,'%')

np.set_printoptions(threshold=200)
result = confusion_matrix(y_test, y_pred2)
print('\nConfusion Matrix:')
print(result)

Accuracy: 40.44750430292599 %

Confusion Matrix:
[[   6  174  342 ...    0    0    0]
 [   1  915  198 ...    0    0    0]
 [   9  418 1586 ...    0    0    0]
 ...
 [   0    1    2 ...    0    0    0]
 [   0    0    2 ...    0    0    0]
 [   0    0    4 ...    0    0    0]]


### **LSTM**

In [ ]:
X=X_train
y=y_train
X=np.expand_dims(X,-1)
print(X.shape)
print(y.shape)

(19170, 3, 1)
(19170,)


In [ ]:
y=np.array(y)
y=y.reshape(-1, 1)
y.shape

(19170, 1)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
y = enc.fit_transform(y)
y=y.toarray()

In [ ]:
#preprocessing test data
X_test=np.expand_dims(X_test,-1)

ytest=np.array(y_test)
ytest=ytest.reshape(-1, 1)
enc = OneHotEncoder()
ytest = enc.fit_transform(ytest)
ytest=ytest.toarray()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

model=Sequential([
      LSTM(256,return_sequences=False, input_shape=(3,1)),
      Dense(128, activation='relu'),
      Dropout(0.2),
      Dense(64, activation='relu'),
      Dropout(0.2),
      Dense(16, activation='softmax'),
])

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 256)               264192    
                                                                 
 dense_9 (Dense)             (None, 128)               32896     
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 dense_10 (Dense)            (None, 64)                8256      
                                                                 
 dropout_7 (Dropout)         (None, 64)                0         
                                                                 
 dense_11 (Dense)            (None, 16)                1040      
                                                                 
Total params: 306,384
Trainable params: 306,384
Non-tr

In [ ]:
history= model.fit(X, y, validation_split=0.2, epochs=100, batch_size=512, shuffle=True)

Epoch 1/100
30/30 [==============================] - 6s 109ms/step - loss: 2.1938 - accuracy: 0.2868 - val_loss: 1.8454 - val_accuracy: 0.3717
Epoch 2/100
30/30 [==============================] - 3s 89ms/step - loss: 1.8646 - accuracy: 0.3604 - val_loss: 1.7905 - val_accuracy: 0.3826
Epoch 3/100
30/30 [==============================] - 3s 90ms/step - loss: 1.8084 - accuracy: 0.3827 - val_loss: 1.7746 - val_accuracy: 0.3811
Epoch 4/100
30/30 [==============================] - 3s 89ms/step - loss: 1.7862 - accuracy: 0.3884 - val_loss: 1.7669 - val_accuracy: 0.3808
Epoch 5/100
30/30 [==============================] - 3s 89ms/step - loss: 1.7660 - accuracy: 0.3871 - val_loss: 1.7638 - val_accuracy: 0.3751
Epoch 6/100
30/30 [==============================] - 3s 92ms/step - loss: 1.7563 - accuracy: 0.3959 - val_loss: 1.7497 - val_accuracy: 0.3753
Epoch 7/100
30/30 [==============================] - 3s 92ms/step - loss: 1.7429 - accuracy: 0.3934 - val_loss: 1.7446 - val_accuracy: 0.3756
Epoch

In [ ]:
scores = model.evaluate(X_test, ytest)
LSTM_accuracy = scores[1]*100
print('Test accuracy: ', scores[1]*100, '%')

200/200 [==============================] - 1s 6ms/step - loss: 1.6816 - accuracy: 0.4059
Test accuracy:  40.58832824230194 %


In [ ]:
# save the model to disk
import pickle
filename = 'LSTM_model.sav'
pickle.dump(model, open(filename, 'wb'))

# some time later...

# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.evaluate(X_test,ytest)
print(result)

200/200 [==============================] - 3s 12ms/step - loss: 1.6857 - accuracy: 0.4049
[1.6856638193130493, 0.40494444966316223]


# **New iMobile Jobs Data**

In [ ]:
raw_csv_data=pd.read_csv("/content/drive/MyDrive/datasetW4/cleanedMobile.csv")
df=raw_csv_data.copy()
df.head(5)

,Job,SenderCity,ReceiverCity,Service,CreatedDate,ReadyDate,DeliveryDate,Days
0,ECC0048484,Christchurch,Christchurch,Economy,05/12/2022,5/12/2022,6/12/2022,NaN
1,ECC0048475,Auckland,Auckland,Metro,05/12/2022,5/12/2022,6/12/2022,NaN
2,ECC0048472,Christchurch,Dunedin,Economy,05/12/2022,5/12/2022,6/12/2022,NaN
3,ECC0048462,Auckland,Kawerau,Economy,05/12/2022,5/12/2022,6/12/2022,NaN
4,ECC0048460,Auckland,Whangarei,SuperEconomy,05/12/2022,5/12/2022,6/12/2022,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Job           8523 non-null   object 
 1   SenderCity    8523 non-null   object 
 2   ReceiverCity  8523 non-null   object 
 3   Service       8523 non-null   object 
 4   CreatedDate   8523 non-null   object 
 5   ReadyDate     8523 non-null   object 
 6   DeliveryDate  8523 non-null   object 
 7   Days          0 non-null      float64
dtypes: float64(1), object(7)
memory usage: 532.8+ KB


In [ ]:
Ddata2=df.copy()

Ddata2['SenderCity'] = Ddata2['SenderCity'].str.lower()
Ddata2['ReceiverCity'] = Ddata2['ReceiverCity'].str.lower()
Ddata2['Service'] = Ddata2['Service'].str.lower()


Ddata2['SenderCity'] = Ddata2['SenderCity'].str.strip()
Ddata2['ReceiverCity'] = Ddata2['ReceiverCity'].str.strip()
Ddata2['Service'] = Ddata2['Service'].str.strip()
Ddata2['ReadyDate'] = Ddata2['ReadyDate'].str.strip()
Ddata2['DeliveryDate'] = Ddata2['DeliveryDate'].str.strip()

Ddata3 = Ddata2.copy()
Ddata2.head(5)

,Job,SenderCity,ReceiverCity,Service,CreatedDate,ReadyDate,DeliveryDate,Days
0,ECC0048484,christchurch,christchurch,economy,05/12/2022,5/12/2022,6/12/2022,NaN
1,ECC0048475,auckland,auckland,metro,05/12/2022,5/12/2022,6/12/2022,NaN
2,ECC0048472,christchurch,dunedin,economy,05/12/2022,5/12/2022,6/12/2022,NaN
3,ECC0048462,auckland,kawerau,economy,05/12/2022,5/12/2022,6/12/2022,NaN
4,ECC0048460,auckland,whangarei,supereconomy,05/12/2022,5/12/2022,6/12/2022,NaN


In [ ]:
from datetime import datetime
i=0
for date_str in Ddata2['ReadyDate']:
  date_object = datetime.strptime(date_str, '%d/%m/%Y').date()
  Ddata2['ReadyDate'][i] = date_object
  i=i+1

i=0
for date_str in Ddata2['DeliveryDate']:
  date_object = datetime.strptime(date_str, '%d/%m/%Y').date()
  Ddata2['DeliveryDate'][i] = date_object
  i=i+1

In [ ]:
Ddata3=Ddata2.copy()
Ddata2.head(5)

,Job,SenderCity,ReceiverCity,Service,CreatedDate,ReadyDate,DeliveryDate,Days
0,ECC0048484,christchurch,christchurch,economy,05/12/2022,2022-12-05,2022-12-06,NaN
1,ECC0048475,auckland,auckland,metro,05/12/2022,2022-12-05,2022-12-06,NaN
2,ECC0048472,christchurch,dunedin,economy,05/12/2022,2022-12-05,2022-12-06,NaN
3,ECC0048462,auckland,kawerau,economy,05/12/2022,2022-12-05,2022-12-06,NaN
4,ECC0048460,auckland,whangarei,supereconomy,05/12/2022,2022-12-05,2022-12-06,NaN


In [ ]:
Ddata2['DeliveryDate'][3]

datetime.date(2022, 12, 6)

In [ ]:
Ddata3['Days']=Ddata3['DeliveryDate']-Ddata3['ReadyDate']
Ddata3.head(5)

,Job,SenderCity,ReceiverCity,Service,CreatedDate,ReadyDate,DeliveryDate,Days
0,ECC0048484,christchurch,christchurch,economy,05/12/2022,2022-12-05,2022-12-06,1 days
1,ECC0048475,auckland,auckland,metro,05/12/2022,2022-12-05,2022-12-06,1 days
2,ECC0048472,christchurch,dunedin,economy,05/12/2022,2022-12-05,2022-12-06,1 days
3,ECC0048462,auckland,kawerau,economy,05/12/2022,2022-12-05,2022-12-06,1 days
4,ECC0048460,auckland,whangarei,supereconomy,05/12/2022,2022-12-05,2022-12-06,1 days


In [ ]:
Ddata3['Days'][0].days

1

In [ ]:
for i in range(len(Ddata3['Days'])):
  a=Ddata3['Days'][i].days
  print(a)
  Ddata3['Days'][i]=a

In [ ]:
Ddata3['Days'].max()

147

In [ ]:
df_preprocessed=Ddata3.copy()
df_preprocessed.to_csv('imobileData.csv', index=False)

In [ ]:
d=pd.read_csv("/content/drive/MyDrive/datasetW4/imobileData.csv")
d.head(5)

,Job,SenderCity,ReceiverCity,Service,CreatedDate,ReadyDate,DeliveryDate,Days
0,ECC0048484,christchurch,christchurch,economy,05/12/2022,2022-12-05,2022-12-06,1
1,ECC0048475,auckland,auckland,metro,05/12/2022,2022-12-05,2022-12-06,1
2,ECC0048472,christchurch,dunedin,economy,05/12/2022,2022-12-05,2022-12-06,1
3,ECC0048462,auckland,kawerau,economy,05/12/2022,2022-12-05,2022-12-06,1
4,ECC0048460,auckland,whangarei,supereconomy,05/12/2022,2022-12-05,2022-12-06,1


In [ ]:
d.drop(d[d['Days'] > 15].index, inplace = True)
d.drop(d[d['Days'] < 0].index, inplace = True)

In [ ]:
d=d.drop(['CreatedDate'], axis=1)
d=d.drop(['ReadyDate'], axis=1)
d=d.drop(['DeliveryDate'], axis=1)
d=d.drop(['Job'], axis=1)

In [ ]:
d.head(10)

,SenderCity,ReceiverCity,Service,Days
0,christchurch,christchurch,economy,1
1,auckland,auckland,metro,1
2,christchurch,dunedin,economy,1
3,auckland,kawerau,economy,1
4,auckland,whangarei,supereconomy,1
5,auckland,whitianga,economy,1
6,auckland,hamilton,supereconomy,1
7,auckland,whangarei,economy,1
8,christchurch,christchurch,metro,1
9,christchurch,christchurch,metro,1


In [ ]:
Ddata5=d.copy()
d['SenderCity']=d['SenderCity'].map(cityDict)
d['ReceiverCity']=d['ReceiverCity'].map(cityDict)
d['Service']=d['Service'].map(serviceDict)
d.head(10)

,SenderCity,ReceiverCity,Service,Days
0,434.0,434.0,2.0,1
1,50.0,50.0,NaN,1
2,434.0,88.0,2.0,1
3,50.0,371.0,2.0,1
4,50.0,51.0,8.0,1
5,50.0,140.0,2.0,1
6,50.0,288.0,8.0,1
7,50.0,51.0,2.0,1
8,434.0,434.0,NaN,1
9,434.0,434.0,NaN,1


In [ ]:
d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8305 entries, 0 to 8519
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SenderCity    8213 non-null   float64
 1   ReceiverCity  7715 non-null   float64
 2   Service       7771 non-null   float64
 3   Days          8305 non-null   int64  
dtypes: float64(3), int64(1)
memory usage: 324.4 KB
